In [22]:
import numpy as numpy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [23]:
df = pd.read_csv('../week03/data/train.csv')

In [24]:
df = pd.read_csv('../week03/data/train.csv', parse_dates=[0])

In [25]:
df['day']= df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['month'] = df['datetime'].dt.month

In [26]:
#this was actually close to what I initially wrote, but elegant, in theory you should do this on train and test if submitting to kaggle
df.loc[df['weather']==4, 'weather'] = 3




In [27]:
df['weather'].value_counts()

weather
1    7192
2    2834
3     860
Name: count, dtype: int64

In [28]:

from sklearn.model_selection import (train_test_split, cross_val_score)

In [29]:
y = df['count'] # this is the dv
X = df.drop(['count', 'registered', 'casual', 'atemp'],  axis =1) #this now separates the DV to prevent data leakage, there is argument about registered and casual being data leak or not
#though I will say having them boosts my score like 25% on the linear regression
#atemp had no effect to at least 3 digits on my linear regression scores
# X = df.drop(['atemp', ])

In [30]:
#def reduc_weather(df):
 #   return df.replace({4:3})

In [31]:
df['datetime']

0       2011-01-01 00:00:00
1       2011-01-01 01:00:00
2       2011-01-01 02:00:00
3       2011-01-01 03:00:00
4       2011-01-01 04:00:00
                ...        
10881   2012-12-19 19:00:00
10882   2012-12-19 20:00:00
10883   2012-12-19 21:00:00
10884   2012-12-19 22:00:00
10885   2012-12-19 23:00:00
Name: datetime, Length: 10886, dtype: datetime64[ns]

In [32]:
#standard xtrain y train for sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=14)

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer  
# turn categorical values into binary columns, ordinal, bins, and scales
from sklearn.preprocessing import (OneHotEncoder, OrdinalEncoder, KBinsDiscretizer, MinMaxScaler, StandardScaler, PolynomialFeatures)
#column transform
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import (LinearRegression, Lasso, Ridge, ElasticNet)

In [34]:
X_train

,datetime,season,holiday,workingday,weather,temp,humidity,windspeed,day,hour,month
3738,2011-09-06 17:00:00,3,0,1,2,22.14,88,26.0027,6,17,9
3991,2011-09-17 08:00:00,3,0,0,2,18.86,77,16.9979,17,8,9
8608,2012-08-01 01:00:00,3,0,1,1,27.06,83,6.0032,1,1,8
4332,2011-10-12 13:00:00,4,0,1,3,22.14,88,15.0013,12,13,10
9506,2012-09-19 11:00:00,3,0,1,1,24.60,43,16.9979,19,11,9
...,...,...,...,...,...,...,...,...,...,...,...
7526,2012-05-12 23:00:00,2,0,0,1,23.78,56,6.0032,12,23,5
6471,2012-03-06 21:00:00,1,0,1,1,13.12,49,12.9980,6,21,3
2454,2011-06-10 04:00:00,2,0,1,1,27.06,78,11.0014,10,4,6
9484,2012-09-18 13:00:00,3,0,1,2,28.70,74,39.0007,18,13,9


In [35]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('bins', KBinsDiscretizer(n_bins=10))
    
])

In [36]:
column_transformer = ColumnTransformer([
    ('one_encode', OneHotEncoder(),['season', 'holiday', 'workingday', 'weather', 'hour', 'day','month',]), #One Hot Encoder is for categorical/discrete variables to turn them into binary matrices 
    ('pipeline', pipeline, ['temp', 'humidity', 'windspeed',]), #you can add casual and registered, windspeed might be easier to just drop?
    ('pass', 'passthrough'),
       
]
,remainder = "drop")

In [37]:
general_pipeline = Pipeline([
    ('column_transformer', column_transformer),
    ("model", LinearRegression())
    
])

In [38]:
#X_train_transformed = column_transformer.fit_transform(X_train,y_train)


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
m_poly = LinearRegression()
m_ridge = Ridge(alpha=1, random_state=14)
m_lasso = Lasso(alpha=1, random_state=14)
m_elastic_net = ElasticNet(alpha=1, l1_ratio=.5,
                           random_state=14)

In [ ]:
m_lasso.fit(X_train_transformed, y_train) 

In [ ]:
ypred_lasso = m_lasso.predict(X_train_transformed)

In [ ]:
m_lasso.score(X_train_transformed, y_train)

In [ ]:
#I do not get this
#m_lasso.score(X_test, y_test)

In [ ]:
# X_train_pd = pd.DataFrame(X_train_transformed, columns=general_pipeline[:-1].get_feature_names_out())

In [ ]:
#general_pipeline.fit(X_train_pd,y_train)

In [ ]:
X_train_transformed.shape

In [ ]:
#column_transformer.get_feature_names_out()


In [ ]:
column_transformer

In [ ]:
model = LinearRegression()

model.fit(X_train_transformed, y_train)

In [ ]:
general_pipeline.fit(X_train, y_train)

In [ ]:
#niceeee score
general_pipeline.score(X_test, y_test)